# Manually Serve the model 

In notebook '04-04-accident-recognition' we were able to use the retrained model to predict a 'severe' or 'moderate' car accident within an image.  

Next we will determine if we can manually connect with the model from the model server and model pipeline we have created and deployed.  This will be done through an API call.

In [ ]:
# If you did not use the Workbench image designed for this Lab, you can uncomment and run the following line to install the required packages.
#!pip install --no-cache-dir --no-dependencies -r requirements.txt

In [1]:
!pip install ultralytics

from ultralytics import YOLO
from PIL import Image

from remote_infer_rest import ort_v5
from remote process_image import perform_inference


[notice] A new release of pip available: 22.2.2 -> 23.3.2
[notice] To update, run: pip install --upgrade pip


## Enter the inference URL, the model name, the YAML file with your classes

In [2]:
#from model internal:  http://modelmesh-serving.user1:8008
infer_url = 'https://my-first-model-user1.apps.cluster-r89n6.sandbox766.opentlc.com/v2/models/my-first-model/infer'

model_name =  'my-first-model'

classes_file = 'datasets/accident-sample/data.yaml'
                            

## Now set the parameters for the inference

In [3]:
# 1. The image you want to analyze
image_path ='images/carImage1.jpg' # You can replace this with an image you upload

# 2. Confidence threshold, between 0 and 1 (detections with less score won't be retained)
conf = 0.2

# 3. Intersection over Union Threshold, between 0 and 1 (cleanup overlapping boxes)
iou = 0.6

## Launch the inference and show the result

In [ ]:
#yolo8 model inference from Robert

infer = perform_inference(image_path, infer_url)

img, out, result = infer()
print(f'{result}')
print('Predictions:')
print(out)
print('Format: each detection is a float64 array shaped as [top_left_corner_x, top_left_corner_y, bottom_right_corner_x, bottom_right_corner_y, confidence, class_index]')
print('The coordinates are relative to a letterboxed representation of the image of size 640x640')
img  = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
fig  = plt.gcf()
fig.set_size_inches(24, 12)
plt.axis('off')
plt.imshow(img);

In [6]:
#yolo5 model inference from Guillaume

infer=ort_v5(image_path, infer_url, conf, iou, 640, classes_file)
img, out, result = infer()
print(f'{result}')
print('Predictions:')
print(out)
print('Format: each detection is a float64 array shaped as [top_left_corner_x, top_left_corner_y, bottom_right_corner_x, bottom_right_corner_y, confidence, class_index]')
print('The coordinates are relative to a letterboxed representation of the image of size 640x640')
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
fig = plt.gcf()
fig.set_size_inches(24, 12)
plt.axis('off')
plt.imshow(img);

TypeError: list indices must be integers or slices, not str

In [ ]:
#define our predict function and pass the path to our model and car image.
def predict(best_model_path, car_image_path):
    model = YOLO(best_model_path)
    results = model.predict(car_image_path)
    return results[0]

In [ ]:
#In the photo draw boxes listing name, probability around each car (object type)
def draw_boxes_image(result):
    print("inside draw boxes image")
    #Image.fromarray(result.plot()[:,:,::-1])

In [ ]:
#call our predict function
prediction_results = predict("models/best.pt", "images/carImage1.jpg")
Image.fromarray(prediction_results.plot()[:,:,::-1])
#draw_boxes_image(prediction_results, image)

## Extract the Predict function into a Python file
Now that we have created a working function, extract the prediction logic into a standalone python file, prediction.py. Also, make sure requirements.txt is updated with any additional packages you have used and need for prediction.

## Test the function from your Python file
We can make sure the extraction worked properly by loading the function from our prediction.py file and calling it (from our notebook) with the same predict function parameters we used above.

In [ ]:
#after creating prediction.py, place the predict function inside the file.  Then call the predict method from within our notebook

from prediction import predict

#image & model file locations.
car_image = "images/carImage1.jpg"
model_path = "models/best.pt"

#call predict function in prediction.py
prediction_results = predict(model_path, car_image)

#display image with boxes,class and probability
Image.fromarray(prediction_results.plot()[:,:,::-1])